In [ ]:
import numpy as np
from mido import MidiFile
from pyqubo import Array, Constraint, Placeholder
from openjij import SQASampler
import pygame

以下の３つを変更

In [ ]:
title = 'canon' # 曲名(classicフォルダの中にあるもの)

In [3]:
n = 20 # 何個の音符を作成するか

In [4]:
track_num = 1 # エラーが出たら0に変更

データの前処理

In [5]:
path = f'classic/{title}.mid'
midi = MidiFile(path)

index = []
index_fin = []
ori_melody = []
ori_melody_fin = []
for i, msg in enumerate(midi.tracks[track_num]):
    if msg.type == 'note_on':
        if msg.velocity > 0:
            index.append(i)
            ori_melody.append(msg.note)
        if msg.velocity == 0:
            index_fin.append(i)
            ori_melody_fin.append(msg.note)

ori_melody = np.array(ori_melody)
ori_melody_fin = np.array(ori_melody_fin)

note_all = np.unique(ori_melody)
count1 = {}
for i in note_all:
    count1[i] = 0

fin_list = []
count2 = {}

for value1 in ori_melody:
    for i in note_all:
        count2[i] = 0
    for j, value2 in zip(index_fin, ori_melody_fin):
        if value1 == value2:
            if count1[value2] == count2[value2]:
                fin_list.append(j)
                count1[value2] += 1
                break
            else:
                count2[value2] += 1

index_fin = fin_list

In [6]:
len_pitch = 12

rem = ori_melody % len_pitch
fin = rem[len(rem) - 1]
dic = {}
for num, note in enumerate(range(len_pitch)):
    indices = np.array([i for i, x in enumerate(rem) if x == note])
    if indices.size == 0:
        indices = False
    else:
        if note != fin:
            indices = indices + 1
        if note == fin:
            indices = indices[:-1] + 1
    dic[note] = rem[indices]

W = np.zeros(len_pitch**2).reshape(len_pitch, len_pitch)

for i in range(len_pitch):
    for j in range(len_pitch):
        W[i, j] = np.sum(dic[i] == j)

for i in range(len_pitch):
    if np.sum(W, axis=1)[i] == 0:
        continue
    else:
        W[i,:] = W[i, :] / np.sum(W, axis=1)[i]

量子アニーリングの実行

In [ ]:
x = Array.create('x', shape=(n, len_pitch), vartype='BINARY')

P = [i for i in range(len_pitch)]

c1 = np.sum([((1 - np.sum([x[i,j] for j in P])) ** 2) for i in range(n)])
o1 = -1*np.sum([np.sum([np.sum([(W[j1,j2]*x[i,j1]*x[i+1,j2]) for j2 in P]) for j1 in P]) for i in range(n-1)])
o2 = np.sum([((np.sum([(j * x[i,j]) for j in P]) - np.sum([(j * x[i+1,j]) for j in P])) ** 2) for i in range(n-1)])

H = Placeholder("lam1") * Constraint(c1, "c1") + Placeholder("lam2") * o1 + Placeholder("lam3") * o2

feed_dict = {'lam1': 15.0, 'lam2': 15.0, 'lam3': 0.5}

model = H.compile()
qubo, offset = model.to_qubo(feed_dict=feed_dict)
sampler = SQASampler()
sampleset = sampler.sample_qubo(qubo, num_reads=2000)
decoded_sample = model.decode_sampleset(sampleset, feed_dict)
if decoded_sample[0].constraints(only_broken=True):
        print("制約を破っています")
        print(decoded_sample[0].constraints(only_broken=True))

結果の表示

In [ ]:
create = []
for i in range(n):
    for j in range(len_pitch):
        if sampleset.first.sample['x[{}][{}]'.format(i, j)] == 1:
            create.append(j)
            
onkai = {0:'ド', 1:'ド#', 2:'レ', 3:'レ#', 4:'ミ', 5:'ファ', 6:'ファ#', 7:'ソ', 8:'ソ#', 9:'ラ', 10:'ラ#', 11:'シ'}
kekka_oto = []
for i in range(n):
    for j in range(len_pitch):
        if sampleset.first.sample['x[{}][{}]'.format(i, j)] == 1:
            kekka_oto.append(onkai[j])
print(kekka_oto)

音楽再生の準備

In [9]:
seq = []

for i in range(n):
    kari = []
    for j in P:
        if decoded_sample[0].array('x', (i, j)) == 1:
            kari.append(j)
    if len(kari) == 1:
        seq += kari
    elif len(kari) > 0:
        if len(seq) == 0:
            seq.append(kari[0])
        else:
            diff = [abs(x - seq[-1]) for x in kari]
            min_diff = min(diff)
            result = [x for x in kari if abs(x - seq[-1]) == min_diff]
            seq.append(result[0])
    elif len(kari) == 0:
        if len(seq) == 0:
            unique, counts = np.unique(rem, return_counts=True)
            first = np.random.choice(unique, p=counts/sum(counts))
            seq.append(first)
        else:
            seq.append(seq[-1])

In [10]:
n = len(ori_melody) - len(seq)
num_rep = 1

rep = len(ori_melody) // n
new_sequence = []
seq_dict = {}

for i in range(rep//num_rep):

    unique, counts = np.unique(rem, return_counts=True)
    first = np.random.choice(unique, p=counts/sum(counts))

    elements = np.arange(len_pitch)
    sequence = [first]

    for _ in range(n - 1):
        probabilities = W[first, :]
        next_element = np.random.choice(elements, p=probabilities)
        sequence.append(next_element)
        first = next_element

    sequence = np.array(sequence)
    seq_dict[i] = list(sequence + 12 * (ori_melody[0+i*n:n+i*n] // len_pitch))

for _ in range(num_rep):
    for i in range(rep//num_rep):
        new_sequence += seq_dict[i]

first = np.random.choice(unique, p=counts/sum(counts))
sequence = [first]
num_lack = len(ori_melody) - len(new_sequence)

for _ in range((num_lack) - 1):
    next_element = np.random.choice(elements, p=probabilities)
    sequence.append(next_element)
    first = next_element

sequence = np.array(sequence)
new_melody = new_sequence + list(sequence + 12 * (ori_melody[-num_lack:] // len_pitch))

In [11]:
melody_dict = {}
for i, note in zip(index, new_melody):
    melody_dict[i] = note

melody_dict_fin = {}
for i, note in zip(index_fin, new_melody):
    melody_dict_fin[i] = note

for i, msg in enumerate(midi.tracks[track_num]):
    if msg.type == 'note_on':
        if msg.velocity > 0:
            msg.note = melody_dict[i]
        if msg.velocity == 0:
            msg.note = melody_dict_fin[i]

In [12]:
np_ind = np.array(index)
ind = np_ind[(ori_melody / len_pitch) >= 6.0]

new_midi = MidiFile()
new_midi.type = midi.type
new_midi.ticks_per_beat = midi.ticks_per_beat
new_midi.tracks = midi.tracks

for i, msg in enumerate(new_midi.tracks[track_num]):
    if i in ind:
        msg.velocity = 0

音楽再生

In [ ]:
path = 'output.mid'
new_midi.save(path)
pygame.init()
pygame.mixer.init()
pygame.mixer.music.load(path)
pygame.mixer.music.play()
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)